In [1]:
import pandas as pd

prot_df = pd.read_csv('data/train_proteins.csv')
pivoted = prot_df.pivot(index="visit_id", columns="UniProt", values="NPX")

In [3]:
pivoted.isna().sum(axis=1)

visit_id
10053_0     62
10053_12    56
10053_18    19
10138_12    10
10138_24     8
            ..
8699_24     11
942_12      15
942_24      10
942_48      11
942_6       11
Length: 1113, dtype: int64

In [7]:
pivoted.columns

Index(['O00391', 'O00533', 'O00584', 'O14498', 'O14773', 'O14791', 'O15240',
       'O15394', 'O43505', 'O60888',
       ...
       'Q9HDC9', 'Q9NQ79', 'Q9NYU2', 'Q9UBR2', 'Q9UBX5', 'Q9UHG2', 'Q9UKV8',
       'Q9UNU6', 'Q9Y646', 'Q9Y6R7'],
      dtype='object', name='UniProt', length=227)

In [2]:
import numpy as np



# Normalize each column separately
for col in pivoted.columns:
    col_data = pivoted[col].dropna()
    mean = col_data.mean()
    std = col_data.std(ddof=1)
    pivoted[col] = (pivoted[col] - mean) / std



# Calculate mean and standard deviation of updrs_1 through updrs_4
updrs_mean = clin_df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].mean()
updrs_std = clin_df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']].std()

norm_clin_df = pd.DataFrame()
# Normalize updrs_1 through updrs_4 and round to 3 sig figs
norm_clin_df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']] = ((clin_df[['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']] - updrs_mean) / updrs_std).round(3)

In [11]:
# Merge prot_df and clin_df on visit_id
merged_df = pd.merge(clin_df, pivoted, on='visit_id')


In [19]:
na_counts = merged_df.isna().sum(axis=0)

In [22]:
print(na_counts[na_counts>0])

updrs_3     10
updrs_4    499
O00391     332
O00533       1
O00584      13
          ... 
Q9UBX5      13
Q9UKV8     198
Q9UNU6     335
Q9Y646      60
Q9Y6R7      68
Length: 202, dtype: int64


In [6]:
## interpolate instead ?
for patient in merged_df.patient_id.unique():
    # Select the patient dataset and make a copy
    patient_data = merged_df[merged_df.patient_id == patient].copy()

    # Fill the NaN values with forward fill, backward fill, and zero
    patient_data.fillna(method='ffill', inplace=True)
    patient_data.fillna(method='bfill', inplace=True)
    patient_data.fillna(0, inplace=True)

    merged_df[merged_df['patient_id']==patient] = patient_data

In [ ]:
import amp_pd_peptide
env = amp_pd_peptide.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test files
for (test, test_peptides, test_proteins, sample_submission) in iter_test:
    sample_prediction_df['rating'] = np.arange(len(sample_prediction))  # make your predictions here
    env.predict(sample_prediction_df)   # register your predictions